In [1]:
import os
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/aleksaDam997/ds_resume_light_project.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "aleksaDam997"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "5f1137a6498c801820d5b434ab2bf66df0bab215"

In [2]:
%pwd

'c:\\Users\\Korisnik\\Desktop\\ml_projects\\ds_resume_light_project\\ds_resume_light_project\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Korisnik\\Desktop\\ml_projects\\ds_resume_light_project\\ds_resume_light_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.ds_resume_light_project.constants import *
from src.ds_resume_light_project.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
      
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = Path(config.root_dir),
            test_data_path = Path(config.test_data_path),
            model_path = Path(config.model_path),
            all_params = params,
            metric_file_name = Path(config.metric_file_name),
            target_column = schema.TARGET_COLUMN,
            mlflow_uri = "https://dagshub.com/aleksaDam997/ds_resume_light_project.mlflow"
        )

        return model_evaluation_config

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [8]:
class modelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_X = test_data.drop(columns=[self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_X)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path = self.config.metric_file_name, data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name = "ElasticNetModel")
            else:
               mlflow.sklearn.log_model(model, "model")

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = modelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-26 23:29:03,022: INFO: common]: YAML file: config\config.yaml loaded successfully
[2026-01-26 23:29:03,029: INFO: common]: YAML file: params.yaml loaded successfully
[2026-01-26 23:29:03,034: INFO: common]: YAML file: schema.yaml loaded successfully
[2026-01-26 23:29:03,036: INFO: common]: Directory created at: artifacts
[2026-01-26 23:29:03,045: INFO: common]: Directory created at: artifacts/model_evaluation


c:\Users\Korisnik\Desktop\ml_projects\ds_resume_light_project\ds_resume_light_project\venv\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[2026-01-26 23:29:03,938: INFO: common]: JSON file saved at: artifacts\model_evaluation\metric.json


2026/01/26 23:29:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'ElasticNetModel'.
2026/01/26 23:29:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run secretive-koi-308 at: https://dagshub.com/aleksaDam997/ds_resume_light_project.mlflow/#/experiments/0/runs/2f6cbc60f4be4ca7b614fd97572b192a
🧪 View experiment at: https://dagshub.com/aleksaDam997/ds_resume_light_project.mlflow/#/experiments/0
